# **Importing Necessary Libraries**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
raw_train_data =pd.read_csv("../input/zillow-prize-1/train_2016_v2.csv",parse_dates=["transactiondate"])
raw_train_data.head()

In [ ]:
house_data = pd.read_csv("../input/zillow-prize-1/properties_2016.csv")
house_data.head()

In [ ]:
def get_unique(name,data_series):
    print("{} has total {} records and {} are unique.".format(name,len(data_series),len(data_series.unique())))

In [ ]:
get_unique("raw_train_data",raw_train_data["parcelid"])

In [ ]:
get_unique("house_data",house_data["parcelid"])

# **Train data has some duplicate values Let's analyse those duplicate values to get better understanding.**

In [ ]:
duplicateRowsDF = raw_train_data[raw_train_data.duplicated(["parcelid"],keep=False)]
print("All Duplicate Rows based on all columns are :")
pd.DataFrame(duplicateRowsDF.head(10))

In [ ]:
duplicateRowsDF["parcelid"].value_counts()

In [ ]:
duplicateRowsDF.loc[duplicateRowsDF["parcelid"]==11842707]

## **Here, we can observe that some houses were sold earlier in the year 2016 and after some month they were sold again in the same year. so we will consider the last selling price of this type of house.**

In [ ]:
unique_train_data = raw_train_data.sort_values("transactiondate").drop_duplicates("parcelid",keep = "last")

In [ ]:
get_unique("unique_train_data",unique_train_data["parcelid"])

##Merging two dataset for modelling

In [ ]:
full_data = pd.DataFrame()
full_data = pd.merge(house_data,unique_train_data,on="parcelid",how="left")

In [ ]:
get_unique("full_data",full_data["parcelid"])

In [ ]:
full_data["logerror"].value_counts().sum()

## **Removing all the rows with null values in target variable since they can not be used in model building.**

In [ ]:
train_data = full_data[full_data["logerror"].notna()]

In [ ]:
len(train_data)

## **Dropping columns with missing value more than 30 percent.**

In [ ]:
def drop_columns(data_series):
    missing_value_df = pd.DataFrame((data_series.isnull().sum()/len(data_series))*100,columns=["missing_value"])
    drop_columns_list = missing_value_df.loc[missing_value_df["missing_value"]>30].index.to_list()
    return drop_columns_list

In [ ]:
drp_col = drop_columns(train_data)

In [ ]:
drp_col

In [ ]:
train_data.drop(columns=drp_col,inplace = True)

In [ ]:
(train_data.isnull().sum()/len(train_data))*100

In [ ]:
train_data.dtypes

In [ ]:
train_data.drop(columns=["transactiondate"],inplace=True)

In [ ]:
train_data.set_index('parcelid',inplace= True)

In [ ]:
train_data.head()

In [ ]:
X = train_data.drop(columns = ['logerror'],inplace=False)
y = pd.DataFrame(train_data['logerror'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
X_train[X_train.columns] = X_train[X_train.columns].apply(pd.to_numeric, errors='coerce')

In [ ]:
X_train = X_train.fillna(X_train.median())

In [ ]:
X_test[X_test.columns] = X_test[X_test.columns].apply(pd.to_numeric, errors='coerce')

In [ ]:
X_test = X_test.fillna(X_test.median())

In [ ]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, y_train)
y_pred = reg.predict(X_test)

In [ ]:
from sklearn import metrics
print(metrics.mean_absolute_error(y_test,y_pred))